In [ ]:
import time
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn import discriminant_analysis, ensemble, linear_model, naive_bayes, neighbors, svm, tree

In [ ]:
from ipynb.fs.full.ClassificationPerformanceIndexes import classificationPerformanceIndexes

## Feature Classification

### SVM

In [ ]:
def SVM(X_train, y_train, X_test, y_test, results):
    clf = svm.SVC()
    svm_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['SVM', :] = classificationPerformanceIndexes (y_test, svm_ind)

In [ ]:
def SVM_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = svm.SVC()
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        y_train = X.loc[train,'seizure']
        X_test = X.iloc[test,:X.shape[1]-1]
        y_test = X.loc[test,'seizure']
        svm_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, svm_ind)
    results.loc['SVM Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteSVM(train_dat, test_dat, train_ind, test_ind, results, features, kf, perfInd):
    print('Implementing SVM method...')
    start = time.time()
    SVM(train_dat, train_ind, test_dat, test_ind, results)
    SVM_Kfold(features, kf, perfInd, results)
    end = time.time()
    print('SVM finished in', round(end - start,2), 'sec\n')

### K-NN

In [ ]:
def KNN(X_train, y_train, X_test, y_test, n_neighbors,results):
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    knn_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['KNN', :] = classificationPerformanceIndexes (y_test, knn_ind)

In [ ]:
def KNN_Kfold(X, n_neighbors, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        y_train = X.loc[train,'seizure']
        X_test = X.iloc[test,:X.shape[1]-1]
        y_test = X.loc[test,'seizure']
        knn_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, knn_ind)
    results.loc['KNN Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteKNN(train_dat, test_dat, train_ind, test_ind, results, knn_neighbors, features, kf, perfInd):
    print('Implementing KNN method...')
    start = time.time()
    KNN(train_dat, train_ind, test_dat, test_ind, knn_neighbors, results)
    KNN_Kfold(features, knn_neighbors, kf, perfInd, results)
    end = time.time()
    print('KNN finished in', round(end - start,2),'sec\n')

### Naive Bayes

In [ ]:
def NaiveBayes(X_train, y_train, X_test, y_test, results):
    clf = naive_bayes.GaussianNB()
    nb_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['Naive Bayes', :] = classificationPerformanceIndexes (y_test, nb_ind)

In [ ]:
def NaiveBayes_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = naive_bayes.GaussianNB()
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        y_train = X.loc[train,'seizure']
        X_test = X.iloc[test,:X.shape[1]-1]
        y_test = X.loc[test,'seizure']
        nb_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, nb_ind)
    results.loc['Naive Bayes Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteNB(train_dat, test_dat, train_ind, test_ind, results, features, kf, perfInd):
    print('Implementing Naive Bayes method...')
    start = time.time()
    NaiveBayes(train_dat, train_ind, test_dat, test_ind, results)
    NaiveBayes_Kfold(features, kf, perfInd, results)
    end = time.time()
    print('Naive Bayes finished in', round(end - start,2),'sec\n')

### Decision Trees

In [ ]:
def DecisionTrees(X_train, y_train, X_test, y_test, results):
    clf = tree.DecisionTreeClassifier()
    dt_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['Decision Trees', :] = classificationPerformanceIndexes (y_test, dt_ind)

In [ ]:
def DecisionTrees_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = tree.DecisionTreeClassifier()
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        y_train = X.loc[train,'seizure']
        X_test = X.iloc[test,:X.shape[1]-1]
        y_test = X.loc[test,'seizure']
        dt_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, dt_ind)
    results.loc['Decision Trees Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteDT(train_dat, test_dat, train_ind, test_ind, results, features, kf, perfInd):
    print('Implementing Decision Trees method...')
    start = time.time()
    DecisionTrees(train_dat, train_ind, test_dat, test_ind, results)
    DecisionTrees_Kfold(features, kf, perfInd, results)
    end = time.time()
    print('Decision Trees finished in', round(end - start,2),'sec\n')

### Random Forest

In [ ]:
def RandomForest(X_train, y_train, X_test, y_test, rf_estim, results):
    clf = ensemble.RandomForestClassifier(n_estimators = rf_estim)
    rf_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['Random Forest', :] = classificationPerformanceIndexes (y_test, rf_ind)

In [ ]:
def RandomForest_Kfold(X, rf_estim, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = ensemble.RandomForestClassifier(n_estimators = rf_estim)
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        y_train = X.loc[train,'seizure']
        X_test = X.iloc[test,:X.shape[1]-1]
        y_test = X.loc[test,'seizure']
        rf_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, rf_ind)
    results.loc['Random Forest Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteRF(train_dat, test_dat, train_ind, test_ind, results, rf_estimators, features, kf, perfInd):
    print('Implementing Random Forest method...')
    start = time.time()
    RandomForest(train_dat, train_ind, test_dat, test_ind, rf_estimators, results)
    RandomForest_Kfold(features, rf_estimators, kf, perfInd, results)
    end = time.time()
    print('Random Forest finished in', round(end - start,2),'sec\n')

### LDA

In [ ]:
def LDA(X_train, y_train, X_test, y_test, results):
    clf = discriminant_analysis.LinearDiscriminantAnalysis(solver= 'eigen')
    lda_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['LDA', :] = classificationPerformanceIndexes (y_test, lda_ind)

In [ ]:
def LDA_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    clf = discriminant_analysis.LinearDiscriminantAnalysis(solver= 'eigen')
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        y_train = X.loc[train,'seizure']
        X_test = X.iloc[test,:X.shape[1]-1]
        y_test = X.loc[test,'seizure']
        lda_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, lda_ind)
    results.loc['LDA Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteLDA(train_dat, test_dat, train_ind, test_ind, results, features, kf, perfInd):
    print('Implementing LDA method...')
    start = time.time()
    LDA(train_dat, train_ind, test_dat, test_ind, results)
    LDA_Kfold(features, kf, perfInd, results)
    end = time.time()
    print('LDA finished in', round(end - start,2),'sec\n')

### Logistic Regression

In [ ]:
def LogisticRegression(X_train, y_train, X_test, y_test, results, max_iter):
    clf = linear_model.LogisticRegression(max_iter = max_iter)
    lr_ind = clf.fit(X_train, y_train).predict(X_test)
    results.loc['Logistic Regression', :] = classificationPerformanceIndexes (y_test, lr_ind)

In [ ]:
def LogisticRegression_Kfold(X, kf, cols, results, max_iter):
    f = pd.DataFrame(columns = cols)
    clf = linear_model.LogisticRegression(max_iter = max_iter)
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        y_train = X.loc[train,'seizure']
        X_test = X.iloc[test,:X.shape[1]-1]
        y_test = X.loc[test,'seizure']
        lr_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, lr_ind)
    results.loc['Logistic Regression Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteLR(train_dat, test_dat, train_ind, test_ind, results, max_iter, features, kf, perfInd):
    print('Implementing Logistic Regression method...')
    start = time.time()
    LogisticRegression(train_dat, train_ind, test_dat, test_ind, results, max_iter)
    LogisticRegression_Kfold(features, kf, perfInd, results, max_iter)
    end = time.time()
    print('Logistic Regression finished in', round(end - start,2),'sec\n')

### LSTM

In [ ]:
def Lstm1Model(unitsLSTM, unitsDense1, unitsDense2, dropoutperc, epoch, lossfunction):
    model = Sequential()
    model.add(LSTM(unitsLSTM))
    model.add(Dropout(dropoutperc))
    model.add(Dense(unitsDense1, activation = 'relu'))
    model.add(Dense(unitsDense2, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = lossfunction, metrics = ['accuracy'])
    return model

In [ ]:
def Lstm2Model(unitsLSTM, unitsDense1, unitsDense2, dropoutperc, epoch, lossfunction):
    model = Sequential()
    model.add(LSTM(unitsLSTM, return_sequences = True))
    model.add(Dropout(dropoutperc))
    model.add(LSTM(unitsLSTM))
    model.add(Dropout(dropoutperc))
    model.add(Dense(unitsDense1, activation = 'relu'))
    model.add(Dense(unitsDense2, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = lossfunction, metrics = ['accuracy'])
    return model

In [ ]:
def LSTM_method (model, batch, epochs, X_train, y_train, X_test, y_test, results, tag):
    model.fit(X_train, y_train, batch_size = batch, epochs = epochs, verbose = 0)
    lstm_ind = (model.predict(X_test, batch_size = batch)>=0.5).astype('int')
    results.loc[tag, :] = classificationPerformanceIndexes (y_test, np.reshape(lstm_ind, lstm_ind.shape[0]))

In [ ]:
def LSTM_method_Kfold(X, kf, cols, model, batch, epochs, results, tag):
    f = pd.DataFrame(columns = cols)
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
        y_train = X.loc[train,'seizure'].values.astype(int)
        X_test = X.iloc[test,:X.shape[1]-1]
        X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
        y_test = X.loc[test,'seizure'].values.astype(int)
        model.fit(X_train, y_train, batch_size = batch, epochs = epochs, verbose = 0)
        lstm_ind = (model.predict(X_test, batch_size = batch)>=0.5).astype('int')
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, np.reshape(lstm_ind, lstm_ind.shape[0]))
    results.loc[tag + ' Kfold', :] = np.array(f.mean(axis=0))

In [ ]:
def CompleteLSTM(train_dat, test_dat, train_ind, test_ind, results, features, kf, perfInd, lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function, batch):
    print('Implementing LSTM method...')
    X_train = np.reshape(train_dat.values, (train_dat.shape[0], 1, train_dat.shape[1]))
    y_train = train_ind.values.astype(int)
    X_test = np.reshape(test_dat.values, (test_dat.shape[0], 1, test_dat.shape[1]))
    y_test = test_ind.values.astype(int)

    print('Starting LSTM for Model 1...')
    start = time.time()
    lstm_model1 = Lstm1Model(lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function)
    LSTM_method (lstm_model1, batch, epochs, X_train, y_train, X_test, y_test, results, 'LSTM1')
    LSTM_method_Kfold (features, kf, perfInd, lstm_model1, batch, epochs, results, 'LSTM1')
    end = time.time()
    print('LSTM for Model 1 finished in', round(end - start,2),'sec')
    
    print('Starting LSTM for Model 2...')
    start = time.time()
    lstm_model2 = Lstm2Model(lstm_units, dense1_units, dense2_units, dropout, epochs, loss_function)
    LSTM_method (lstm_model2, batch, epochs, X_train, y_train, X_test, y_test, results, 'LSTM2')
    LSTM_method_Kfold (features, kf, perfInd, lstm_model2, batch, epochs, results, 'LSTM2')
    end = time.time()
    print('LSTM for Model 2 finished in', round(end - start,2),'sec')